# AdaBoost (Adaptive Boosting)

**AdaBoost通过在每轮降低分对样本的权重，增加分错样本的权重，使得分类器在迭代中逐步改进，最终将所有分类器线性组合得到最终分类器.**

![](AdaBoosting.png)

## 1. Boosting 

Boosting 指的是一类集成方法，其主要思想就是将弱的基学习器提升（Boost）为强学习器。具体步骤如下：

1. 先用每个样本权重相等的训练集训练一个初始的基学习器；
2. 根据上轮得到的学习器对训练集的预测表现情况调整训练集中的样本权重（例如提高被错分类的样本的权重使之在下轮训练中得到更多的关注），然后据此训练一个新的基学习器；
3. 重复2直到得到M个基学习器，最终的集成结果是M个基学习器的组合。


## 2. AdaBoost原理

1. 如何调整每一轮的训练集中的样本权重？
2. 如何将得到的M个组合成最终的学习器？

AdaBoost算法采取的方法是：

1. 提高上一轮被错误分类样本的权重，降低被正确分类的样本的权重；
2. 线性加权求和。误差率小的基学习器拥有较大的权值，误差率大的基学习器拥有较小的权值。

## 2.1 算法步骤

考虑二分类问题，训练集：$\{(x_1,y_1),(x_2,y_2),...,(x_n,y_n)\}$，其中$x_i$是一个含有$d$个元素的列向量，即$x_i\in X \subseteq R^d$，$y_i$是标量，$y_i\in \{+1,-1\}$。

1. 初始化样本的权重
$$D_1=(w_{11},w_{12},...,w_{1N}), w_{1i}=\frac{1}{N}, i=1,2,...,N$$

2. 对$m=1,2,...,M$，重复一下操作得到$M$个基学习器：
    1. 按照样本权重分布$D_m$训练数据得到第$m$个基学习器$G_m(x)$:
    $$G_m(x): X \rightarrow \{-1,+1\}$$
    2. 计算$G_m(x)$在加权训练数据集上的**分类误差率：**
    $$e_m=\sum_{i=1}^N{P(G_m(x_i)\neq y_i)}=\sum_{i=1}^N{w_{mi}I(G_m(x_i)\neq y_i)}$$
    上式中$I(\cdot)$是指示函数，考虑更加周全的AdaBoost算法在这一步还应该判断时候满足基本条件（例如生成的      基学习器时候比随机猜测好），如果不满足，则当前基学习器被抛弃，学习过程提前终止。
    3. 计算$G_m(x)$系数（即最终集成使用的基学习器的权重）：
    $$\alpha_m=\frac{1}{2}log \frac{1-e_m}{e_m}$$
    4. 更新训练样本的权重
    $$D_{m+1}=(w_{m+1,1},w_{m+1,2},...,w_{m+1,N})$$
    $$w_{m+1,i}=\frac{w_{mi}}{Z_m}exp(-\alpha_m y_i G_m(x_i)), i=1,2,...,N$$
    其中$Z_m$是规范化因子，目的是为了使$D_{m+1}$的所有元素和为1.即
    $$Z_m=\sum_{i=1}^N{w_{mi}exp(-\alpha_m y_i G_m(x_i))}$$
3. 构建最终的分类器线性组合
$$f(x)=\sum_{i=1}^M{\alpha_m G_m(x)}$$
得到最终的分类器为
$$G(x)=sign(f(x))=sign(\sum_{i=1}^M\alpha_m G_m(x))$$

## 3. AdaBoost算法解释

AdaBoost算法的最终模型表达式为：
$$f(x)=\sum_{i=1}^M{\alpha_mG_m(x)}$$

我们希望这个模型在训练集上的经验误差最小，即
$$\min \sum_{i=1}^N{L(y_i,,f(x))} \Leftrightarrow \min \sum_{i=1}^N{L(y_i, \sum_{i=1}^M{\alpha_mG_m(x)})}$$
$$\Leftrightarrow$$
$$\min \sum_{i=1}^N{L(y_i, f_{m-1}(x) + {\alpha_mG_m(x)})}$$

AdaBoost的损失函数采用**指数损失函数**.

>> 指数损失函数即$L(y,f(x))=exp(-yf(x))$周志华《机器学习》p174有证明，指数损失函数是分类任务原本0/1损失函数的一致(consistent)替代损失函数，由于指数损失函数有更好的数学性质，例如处处可微，所以我们用它替代0/1损失作为优化目标。

将指数损失函数带入目标函数：
$$\arg\min_{\alpha_m,G_m} \sum_{i=1}^N{exp[-y_i(f_{m-1}(x_i)+\alpha_mG_m(x_i))]}$$

因为$y_if_{m-1}(x)$与优化变量$\alpha,G$无关，如果令
$$w_{m,i}=exp[-y_if_{m-1}(x_i)]$$
则有
$$\arg\min_{\alpha_m,G_m} \sum_{i=1}^N{w_{m,i}exp(-\alpha_mG_m(x_i))}$$

### 求解最优解 $\hat{\alpha_m}$和$\hat{G}_m(x)$

1. 固定$\alpha_m>0$，求$\hat{G}_m(x)$:
$$\hat{G}_m(x)=\arg \min_{G_m} \sum_{i=1}^N{w_{m,i}I(y_i\neq G_m(x_i))}$$
上式将指数函数换成指示函数是因为前面说的指数损失函数和0/1损失函数是一致等价的。优化这个式子其实就等于基学习器的学习过程。

2. 求解$\hat{\alpha_m}$

$$\sum_{i=1}^N{w_{m,i}exp(-y_i\alpha_m\hat{G}_m(x))}=\sum_{y_i=\hat{G}_m(x_i)}{w_{m,i}e^{\alpha_m}} + \sum_{y_i\neq G_m(x_i)}{w_{m,i}e^{\alpha_m}}$$
$$=(e^{\alpha_m}-e^{-\alpha_m}) \sum_{i=1}^N{w_{m,i}I(y_i\neq \hat{G}_m(x_i))} + e^{-\alpha_m}\sum_{i=1}^N{w_{m,i}}$$
对$\alpha_m$求导并令其等于0，可得：
$$\hat{\alpha}_m=\frac{1}{2}log \frac{1-e_m}{e_m}$$
其中，$e_m$是分类误差率：
$$e_m=\frac{\sum_{i=1}^N{w_{m,i}I(y_i\neq G_m(x_i))}}{\sum_{i=1}^N{w_{m,i}}}$$

3. 每一轮的权重的更新：
$$w_{m+1,i}=exp[-y_if_m(x)]=exp(-y_i(f_{m-1}(x)+\alpha_mG_m(x)))$$
$$=exp(-y_if_{m-1}(x))\cdot exp(-y_i \alpha_mG_m(x))$$
$$=w_{m,i}exp(-y_i \alpha_mG_m(x))$$

- **AdaBoost算法是一个加性模型，将其简化成前向分步算法求解；**
- **将0/1损失函数用数学性质更好的指数损失函数替代。**

In [ ]:
# Fit a simple decision tree(weak classifier) first
clf_tree = DecisionTreeClassifier(max_depth = 1, random_state = 1)


In [ ]:
def my_adaboost_clf(Y_train, X_train, Y_test, X_test, M=20, weak_clf=DecisionTreeClassifier(max_depth = 1)):
    n_train, n_test = len(X_train), len(X_test)
    # Initialize weights
    w = np.ones(n_train) / n_train
    pred_train, pred_test = [np.zeros(n_train), np.zeros(n_test)]

    for i in range(M):
        # Fit a classifier with the specific weights
        weak_clf.fit(X_train, Y_train, sample_weight = w)
        pred_train_i = weak_clf.predict(X_train)
        pred_test_i = weak_clf.predict(X_test)

        # Indicator function
        miss = [int(x) for x in (pred_train_i != Y_train)]
        print("weak_clf_%02d train acc: %.4f"
         % (i + 1, 1 - sum(miss) / n_train))

        # Error
        err_m = np.dot(w, miss)
        # Alpha
        alpha_m = 0.5 * np.log((1 - err_m) / float(err_m))
        # New weights
        miss2 = [x if x==1 else -1 for x in miss] # -1 * y_i * G(x_i): 1 / -1
        w = np.multiply(w, np.exp([float(x) * alpha_m for x in miss2]))
        w = w / sum(w)

        # Add to prediction
        pred_train_i = [1 if x == 1 else -1 for x in pred_train_i]
        pred_test_i = [1 if x == 1 else -1 for x in pred_test_i]
        pred_train = pred_train + np.multiply(alpha_m, pred_train_i)
        pred_test = pred_test + np.multiply(alpha_m, pred_test_i)

    pred_train = (pred_train > 0) * 1
    pred_test = (pred_test > 0) * 1

    print("My AdaBoost clf train accuracy: %.4f" % (sum(pred_train == Y_train) / n_train))
    print("My AdaBoost clf test accuracy: %.4f" % (sum(pred_test == Y_test) / n_test